In [2]:
import os
import shutil
import sys

import cv2
import numpy as np 
import onnx
import torch
import tensorflow as tf 
from PIL import Image
from torchvision import transforms
from torchvision.models import *
from torchsummary import summary
from onnx_tf.backend import prepare

C:\Users\nguye\.conda\envs\pytorch_envs\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\nguye\.conda\envs\pytorch_envs\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [3]:
import sys
sys.path.append('C:/Users/nguye/OneDrive/Desktop/ai4theblind/Resnet-ssd/')
import numpy as np
import torch
from transform import SSDTransformer
import cv2
from PIL import Image
from Default_boxes import generate_dboxes
from encoder import Encoder
from models import SSD, ResNet
import pyttsx3
import time
import config

[Tham khảo từ bài viết](https://viblo.asia/p/huong-dan-convert-pytorch-sang-tf-lite-YWOZrV8RZQ0)

In [4]:
def convert_torch_to_onnx(onnx_path, image_path, model=None, torch_path=None):
    """
        Coverts Pytorch model file to ONNX
        :param torch_path: Torch model path to load
        :param onnx_path: ONNX model path to save
        :param image_path: Path to test image to use in export progress
    """
    if torch_path is not None:
        pytorch_model = get_torch_model(torch_path)
    else:
        pytorch_model = model
    
    torch_image = get_example_input(image_path)

    torch.onnx.export(
        model = pytorch_model,
        args = torch_image,
        f = onnx_path,
        verbose = False,
        export_params=True,
        do_constant_folding = False,
        input_names = ['input'],
        opset_version = 10,
        output_names = ['output'])

In [5]:
model = SSD(backbone=ResNet())
checkpoint = torch.load(config.pretrained_model)
model.load_state_dict(checkpoint["model_state_dict"])
# if torch.cuda.is_available():
#         model.cuda()
model.eval()
dboxes = generate_dboxes()
transformer = SSDTransformer(dboxes, (300, 300), val=True)
def get_example_input(image_file):
    """
        Load image from disk and converts to compatible shape
        :param image_file: Path to single image file
        :return: Orginal image, numpy.ndarray instance image, torch.Tensor image
    """
    frame = cv2.imread(image_file)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = Image.fromarray(frame)
    frame, _, _, _ = transformer(frame, None, torch.zeros(1, 4), torch.zeros(1))
    image = frame.unsqueeze(dim=0)
#     image = image.cuda()
    return image

In [5]:
model = SSD(backbone=ResNet())
checkpoint = torch.load(config.pretrained_model)
model.load_state_dict(checkpoint["model_state_dict"])
# if torch.cuda.is_available():
#         model.cuda()
model.eval()
dboxes = generate_dboxes()
transformer = SSDTransformer(dboxes, (300, 300), val=True)


onnx_model_path ='model.onnx'
image_path = "../Resnet-ssd/data_test/000104.jpg"

convert_torch_to_onnx(onnx_model_path,image_path, model=model)


In [6]:
def convert_onnx_to_tf(onnx_path, tf_path):
    """
        Converts ONNX model to TF 2.X saved file
        :param onnx_path: ONNX model path to load
        :param tf_path: TF model path to save
    """
    onnx_model = onnx.load(onnx_path)
    onnx.checker.check_model(onnx_model)
    tf_rep = prepare(onnx_model)  #Prepare TF representation
    tf_rep.export_graph(tf_path)  #Export the model

In [7]:
onnx_path ="../Resnet-ssd/model.onnx"
tf_path = "model_tf"

convert_onnx_to_tf(onnx_path, tf_path)


INFO:tensorflow:Assets written to: model_tf\assets


INFO:tensorflow:Assets written to: model_tf\assets


In [ ]:
tf_model_path ="../Resnet-ssd/model_tf/"
image_test_path = "../Resnet-ssd/data_test/000104.jpg"

input_tensor = get_example_input(image_test_path) 

model = tf.saved_model.load(tf_model_path)
model.trainable = False

out = model(**{'input':input_tensor})
print(out)

In [6]:
def convert_tf_to_tflite(tf_path, tf_lite_path):
    """
        Converts TF saved model into TFLite model
        :param tf_path: TF saved model path to load
        :param tf_lite_path: TFLite model path to save
    """
    converter = tf.lite.TFLiteConverter.from_saved_model(tf_path)
    tflite_model  = converter.convert()
    with open(tf_lite_path, 'wb') as f:
        f.write(tflite_model)
        

tf_model_path = "../Resnet-ssd/model_tf"
tflite_model_path ="model.tflite"

convert_tf_to_tflite(tf_model_path, tflite_model_path)
